<a href="https://colab.research.google.com/github/harsha361/pytorch_rnn/blob/main/pytorch_rnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as trasnforms


In [2]:
#device configuration
device=torch.device('cuda'if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [3]:
import torchvision
import torchvision.transforms as transforms
#hyper parameters
sequence_length=28
input_size=28
hidden_size=128
num_layers=2
num_classes=10
batch_size=100
num_epochs=2
learning_rate=0.01
#MNIST dataset and data loader
train_dataset=torchvision.datasets.MNIST(root='/content/sample_data',train=True,transform=transforms.ToTensor(),
                                         download=True)
test_dataset=torchvision.datasets.MNIST(root='/content/sample_data',train=False,transform=transforms.ToTensor())

#dataloader
train_dataloader=torch.utils.data.DataLoader(dataset=train_dataset,batch_size=batch_size,shuffle=True)
test_dataloader=torch.utils.data.DataLoader(dataset=test_dataset,batch_size=batch_size,shuffle=True)


In [4]:
#recurrent neural network(many to one)
class RNN(nn.Module):

  def __init__(self,input_size,hidden_size,num_layers,num_classes):
    super(RNN, self).__init__()
    self.hidden_size=hidden_size
    self.num_layers=num_layers
    self.lstm=nn.LSTM(input_size,hidden_size,num_layers,batch_first=True)
    self.fc=nn.Linear(hidden_size,num_classes)
  def forward(self,x):
    #setting intial and hidden cell states
    h0=torch.zeros(self.num_layers,x.size(0),self.hidden_size).to(device)
    c0=torch.zeros(self.num_layers,x.size(0),self.hidden_size).to(device)
    #forward propagation lstm
    out,_=self.lstm(x,(h0,c0))
    out=self.fc(out[:,-1,:])
    return out

model=RNN(input_size,hidden_size,num_layers,num_classes).to(device)




In [5]:

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Train the model
total_step = len(train_dataloader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_dataloader):
        images = images.reshape(-1, sequence_length, input_size).to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

# Test the model
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_dataloader:
        images = images.reshape(-1, sequence_length, input_size).to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Test Accuracy of the model on the 10000 test images: {} %'.format(100 * correct / total))

# Save the model checkpoint
torch.save(model.state_dict(), 'model.ckpt')

Epoch [1/2], Step [100/600], Loss: 0.4447
Epoch [1/2], Step [200/600], Loss: 0.2529
Epoch [1/2], Step [300/600], Loss: 0.0942
Epoch [1/2], Step [400/600], Loss: 0.2318
Epoch [1/2], Step [500/600], Loss: 0.1947
Epoch [1/2], Step [600/600], Loss: 0.1197
Epoch [2/2], Step [100/600], Loss: 0.1343
Epoch [2/2], Step [200/600], Loss: 0.0651
Epoch [2/2], Step [300/600], Loss: 0.1215
Epoch [2/2], Step [400/600], Loss: 0.0376
Epoch [2/2], Step [500/600], Loss: 0.0987
Epoch [2/2], Step [600/600], Loss: 0.0327
Test Accuracy of the model on the 10000 test images: 97.27 %
